In [1]:
import numpy as np
import pandas as pd

# local package
from full_matrix import gen_full_matrix

In [21]:
VOCAB_SIZE = 2500

TRAIN_DATA_FILE = 'spam-data/train-data.txt'
TEST_DATA_FILE = 'spam-data/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'spam-data/token-spam-prob.txt'
TOKEN_HAM_PROB_FILE = 'spam-data/token-ham-prob.txt'
TOKEN_OVERALL_PROB_FILE = 'spam-data/token-overall-prob.txt'

TEST_FEATURE_MATRIX = 'spam-data/test-features.txt'
TEST_TARGET_FILE = 'spam-data/test-target.txt'

# *Load the data generated from the pre_proccess stage*

In [3]:
%%time

sparse_train_data = np.loadtxt(TRAIN_DATA_FILE, delimiter=' ', dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

# CPU times: user 4.44 s, sys: 60.3 ms, total: 4.5 s
# Wall time: 4.51 s

CPU times: user 4.52 s, sys: 73.8 ms, total: 4.59 s
Wall time: 4.6 s


In [4]:
# quick check the data loaded
print('train:\n', sparse_train_data[:3], '\n---\n',sparse_train_data[-3:])
print('test: \n', sparse_test_data[:3], '\n---\n', sparse_test_data[-3:])

train:
 [[0 1 1 1]
 [0 5 1 1]
 [0 6 1 1]] 
---
 [[5795 2144    0    1]
 [5795 2267    0    1]
 [5795 2303    0    1]]
test: 
 [[8 0 1 2]
 [8 2 1 1]
 [8 3 1 1]] 
---
 [[5793 1872    0    1]
 [5793 1964    0    1]
 [5793 2310    0    1]]


In [5]:
print('Rows in TRAINING file: ', sparse_train_data.shape[0])
print('Emails in TRAINING file: ', np.unique(sparse_train_data[:, 0]).size)
print('\n')
print('Rows in TESTING file: ', sparse_test_data.shape[0])
print('Emails in TESTING file: ', np.unique(sparse_test_data[:, 0]).size)

Rows in TRAINING file:  264056
Emails in TRAINING file:  4016


Rows in TESTING file:  112088
Emails in TESTING file:  1721


---

## *Generate the full matrix from the sparse_train_data*

In [6]:
%%time
full_train_data = gen_full_matrix(sparse_train_data, VOCAB_SIZE)

# CPU times: user 22.6 s, sys: 315 ms, total: 22.9 s
# Wall time: 23.1 s

CPU times: user 14.1 s, sys: 136 ms, total: 14.3 s
Wall time: 14.2 s


In [7]:
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,2,0,2,2,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,2,1,2,2,4,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,2,2,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,6,0,3,3,2,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,2,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


---

### *Probability of being spam*

In [8]:
# probability of being Spam

print('Total documents:', full_train_data.CATEGORY.size)
print('Total of spam:', full_train_data.CATEGORY.sum())

prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of being Spam:', '%.2f' % (prob_spam*100), '%')

Total documents: 4016
Total of spam: 1251
Probability of being Spam: 31.15 %


### *Number of tokens*

In [9]:
# total number of tokens
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY'] # excluding category column

email_lenghts = full_train_features.sum(axis=1) # for each row sum up all columns
email_lenghts

DOC_ID
0       134
1       116
2        34
3       113
4        34
       ... 
5789     88
5790     92
5791     72
5794    113
5795    124
Length: 4016, dtype: int64

In [10]:
total_tokens = email_lenghts.sum()
print('Total number of tokens:', total_tokens)

Total number of tokens: 439447


In [11]:
# total number of tokens in SPAM emails
spam_lengths = email_lenghts[full_train_data.CATEGORY == 1]
total_spam_tokens = spam_lengths.sum()
print('Total number of tokens in SPAM emails:', total_spam_tokens)

Total number of tokens in SPAM emails: 191434


In [12]:
# total number of tokens in HAM emails
ham_lenghts = email_lenghts[full_train_data.CATEGORY == 0]
total_ham_tokens = ham_lenghts.sum()
print('Total number of tokens in HAM emails:', total_ham_tokens)

Total number of tokens in HAM emails: 248013


In [13]:
# sum up each token occurence in spam/ham
spam_train_features = full_train_features.loc[full_train_data.CATEGORY == 1]
summed_spam = spam_train_features.sum(axis=0) + 1 # LaPlace smoothing

ham_train_features = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham = ham_train_features.sum(axis=0) + 1

In [14]:
summed_spam

0       2307
1       1011
2       1363
3       2109
4       1326
        ... 
2495       5
2496       6
2497       2
2498      19
2499       3
Length: 2500, dtype: int64

In [15]:
summed_ham

0       5145
1       2427
2       2022
3        869
4       1624
        ... 
2495      27
2496      21
2497      31
2498      19
2499      31
Length: 2500, dtype: int64

---

# *Calculate the probability that a token occurs given the email is SPAM / HAM, and overall probability*

## *P(Token | Spam)*

In [16]:
prob_token_spam = summed_spam / (total_spam_tokens + VOCAB_SIZE) # LaPlace smoothing balance
print('Check sum:', prob_token_spam.sum())
prob_token_spam

Check sum: 1.0


0       0.011896
1       0.005213
2       0.007028
3       0.010875
4       0.006837
          ...   
2495    0.000026
2496    0.000031
2497    0.000010
2498    0.000098
2499    0.000015
Length: 2500, dtype: float64

## *P(Token | Ham)*

In [17]:
prob_token_ham = summed_ham / (total_ham_tokens + VOCAB_SIZE)  # LaPlace smoothing balance
print('Check sum:', prob_token_ham.sum())
prob_token_ham

Check sum: 1.0


0       0.020538
1       0.009688
2       0.008071
3       0.003469
4       0.006483
          ...   
2495    0.000108
2496    0.000084
2497    0.000124
2498    0.000076
2499    0.000124
Length: 2500, dtype: float64

## *P(Token)*

In [18]:
# overall probability that a token occurs (regardless of spam or nonspam)
prob_token_overall = full_train_features.sum(axis=0) / total_tokens
print('Check sum:', prob_token_overall.sum())
prob_token_overall

Check sum: 1.0


0       0.016953
1       0.007819
2       0.007698
3       0.006772
4       0.006708
          ...   
2495    0.000068
2496    0.000057
2497    0.000071
2498    0.000082
2499    0.000073
Length: 2500, dtype: float64

# *Export the trained model (probabilities) as txt files.*

In [19]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_token_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_token_ham)
np.savetxt(TOKEN_OVERALL_PROB_FILE, prob_token_overall)

---
# *Prepare test data*

In [20]:
%%time
full_test_data = gen_full_matrix(sparse_test_data, VOCAB_SIZE)

# CPU times: user 6.97 s, sys: 39.6 ms, total: 7.01 s
# Wall time: 7.02 s

CPU times: user 6.97 s, sys: 39.6 ms, total: 7.01 s
Wall time: 7.02 s


In [22]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [23]:
# export
np.savetxt(TEST_FEATURE_MATRIX, X_test)
np.savetxt(TEST_TARGET_FILE, y_test)

---